[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/abdn-cs3033-ai/practicals/blob/main/week06/tutorial5-hmm.ipynb)

# CS3033: Artificial Intelligence

## Tutorial 05: Hidden Markov Models

#### Prof. Felipe Meneguzzi

Adapted from code in the [AIMA-Python](https://github.com/aimacode/aima-python) public repository.

In order to run this tutorial, you need to download the auxiliary files from Github into your notebook, which we do with Jupyter's shell commands (if you downloaded the entire repo, the code below is not necessary).

In [ ]:
try:
    import google.colab
    print("We are in Google colab, we need to clone the repo")
    !git clone https://github.com/abdn-cs3033-ai/practicals.git
    %cd practicals/week06
except:
    print("Not in colab")

## Hidden Markov Models

A Markov model is basically a chain-structured Bayesian Network in which there is one state for each time step and each node has an identical probability distribution.
The first node, however, has a different distribution, called the prior distribution which models the initial state of the process.
A state in a Markov model depends only on the previous state and the latest evidence and not on the states before it.


A **Hidden Markov Model** or **HMM** is a special case of a Markov model in which the state of the process is described by a single discrete random variable.
The possible values of the variable are the possible states of the world.


But what if we want to model a process with two or more state variables?
In that case, we can still fit the process into the HMM framework by redefining our state variables as a single "megavariable".
We do this because carrying out inference on HMMs have standard optimized algorithms.
A HMM is very similar to an MDP, but we don't have the option of taking actions like in MDPs, instead, the process carries on as new evidence appears.


If a HMM is truncated at a fixed length, it becomes a Bayesian network and general BN inference can be used on it to answer queries.

Before we start, it will be helpful to understand the structure of a temporal model. We will use the example of the book with the guard and the umbrella. In this example, the state $$\textbf{X}$$ is whether it is a rainy day (`X = True`) or not (`X = False`) at Day $\textbf{t}$. In the sensor or observation model, the observation or evidence $\textbf{U}$ is whether the professor holds an umbrella (`U = True`) or not (`U = False`) on **Day** $\textbf{t}$. Based on that, the transition model is 

| $X_{t-1}$        | $X_{t}$      | **P**$(X_{t} \mid X_{t-1})$ |
| ------------- | ------------- | --------------------------- |
| ***${False}$***  | ***${False}$***  | 0.7                         |
| ***${False}$***  | ***${True}$***   | 0.3                         |
| ***${True}$***   | ***${False}$***  | 0.3                         |
| ***${True}$***   | ***${True}$***   | 0.7                         |

And the the sensor model will be,

| $X_{t}$          | $U_{t}$         | **P**$(U_{t} \mid X_{t})$      |  
| :-------------:  |:-------------:  | :------------------------:|
| ***${False}$***  | ***${True}$***  | 0.2     |
| ***${False}$***  | ***${False}$*** | 0.8     |
| ***${True}$***   | ***${True}$***  | 0.9     |
| ***${True}$***   | ***${False}$*** | 0.1     |

We will implement HMMs in the **`HiddenMarkovModel`** class, below:

In [ ]:
from utils import vector_add, scalar_vector_product, normalize, element_wise_product, matrix_multiplication, T, F

import numpy as np

from notebook import pseudocode

class HiddenMarkovModel:
    """A Hidden markov model which takes Transition model and Sensor model as inputs"""

    def __init__(self, transition_model, sensor_model, prior=None):
        self.transition_model = transition_model
        self.sensor_model = sensor_model
        self.prior = prior or [0.5, 0.5]

    def sensor_dist(self, ev):
        if ev is True:
            return self.sensor_model[0]
        else:
            return self.sensor_model[1]

The class above has attributes for the transition model, the sensor model, and the prior (initialised to a default uniform distribution).

Note that we also import a number of auxiliary functions you will need in implementing the algorithms we require, specifically:

- ```vector_add```
- ```scalar_vector_prodocut```
- ```normalize```
- ```element_wise_product```
- ```matrix_multiplication```

We instantiate the object **`hmm`** of the class using a list of lists for both the transition and the sensor model.


In [ ]:
umbrella_transition_model = [[0.7, 0.3], [0.3, 0.7]]
umbrella_sensor_model = [[0.9, 0.2], [0.1, 0.8]]
hmm = HiddenMarkovModel(umbrella_transition_model, umbrella_sensor_model)

The **`sensor_dist()`** method returns a list with the conditional probabilities of the sensor model.

In [ ]:
hmm.sensor_dist(ev=True)

Now that we have defined an HMM object, our task here is to compute the belief $B_{t}(x)= P(X_{t} \mid U_{1:t})$ given evidence **U** at each time step **t**.


The basic inference tasks that must be solved are:
1. **Filtering**: Computing the posterior probability distribution over the most recent state, given all the evidence up to the current time step.
2. **Prediction**: Computing the posterior probability distribution over the future state.
3. **Smoothing**: Computing the posterior probability distribution over a past state. Smoothing provides a better estimation as it incorporates more evidence.
4. **Most likely explanation**: Finding the most likely sequence of states for a given observation
5. **Learning**: The transition and sensor models can be learnt, if not yet known, just like in an information gathering agent


There are three primary methods to carry out inference in Hidden Markov Models:
1. The Forward-Backward algorithm
2. Fixed lag smoothing
3. Particle filtering (which we don't cover in this course)

Let's have a look at how we can carry out inference and answer queries based on our umbrella HMM using these algorithms.

### Forward-Backward
This is a general algorithm that works for all Markov models, not just HMMs.
In the filtering task (inference) we are given evidence **U** in each time **t** and we want to compute the belief $B_{t}(x)= P(X_{t} \mid U_{1:t})$. 
We can think of it as a three step process:

1. In every step we start with the current belief $P(X_{t}|e_{1:t})$
2. We update it for time
3. We update it for evidence

The forward algorithm performs the step 2 and 3 at once. It updates, or better say reweights, the initial belief using the transition and the sensor model. Let's see the umbrella example. On  **Day 0** no observation is available, and for that reason we will assume that we have equal possibilities to rain or not. In the **`HiddenMarkovModel`** class, the prior probabilities for **Day 0** are by default [0.5, 0.5]. 

#### Forward Message

We can calculate the observation update with the **`forward()`** function. Basically, we update our belief using the observation model. The function returns a list with the probabilities of **raining or not** on **Day 1**.

$$P(X_{t+1} \mid \mathbf{\mathsf{e}}_{1:t+1}) = \alpha P(\mathbf{\mathsf{e}}_{t+1} \mid X_{t+1}) \sum_{x_{t}} P(X_{t+1} \mid x_{t}) P(x_t \mid \mathbf{\mathsf{e}}_{1:t})$$

Which we implement in Python below

In [ ]:
def forward(HMM, fv, ev):
    prediction = vector_add(scalar_vector_product(fv[0], HMM.transition_model[0]),
                            scalar_vector_product(fv[1], HMM.transition_model[1]))
    sensor_dist = HMM.sensor_dist(ev)

    return normalize(element_wise_product(sensor_dist, prediction))

Given the vectorised implementation above, it may be hard to figure out what is the relationship between that code and the formula above. To help you out with that, we highlight what each part of the code above correspond to.

$$P(X_{t+1} \mid \mathbf{\mathsf{e}}_{1:t+1}) = \alpha \underbrace{\underbrace{P(\mathbf{\mathsf{e}}_{t+1} \mid X_{t+1})}_{\mathtt{sensor\_dist}} \underbrace{\sum_{x_{t}} P(X_{t+1} \mid x_{t}) \underbrace{P(x_t \mid \mathbf{\mathsf{e}}_{1:t})}_{\mathtt{fv}}}_{\mathtt{prediction}}}_{\mathtt{element\_wise\_product(sensor\_dist, prediction)}}$$

In [ ]:
umbrella_prior = [0.5, 0.5]
belief_day_1 = forward(hmm, umbrella_prior, ev=True)
print ('The probability of raining on day 1 is {:.2f}'.format(belief_day_1[0]))

In **Day 2** our initial belief is the updated belief of **Day 1**.
Again using the **`forward()`** function we can compute the probability of raining in **Day 2**

In [ ]:
belief_day_2 = forward(hmm, belief_day_1, ev=True)
print ('The probability of raining in day 2 is {:.2f}'.format(belief_day_2[0]))

#### Backward Message

In the smoothing part we are interested in computing the distribution over past states given evidence up to the present. Assume that we want to compute the distribution for the time **k**, for $0\leq k<t $, the computation can be divided in two parts: 
1. The forward message will be computed till and by filtering forward from 1 to **k**.
2. The backward message can be computed by a recusive process that runs from **k** to **t**. 

Rather than starting at time 1, the algorithm starts at time **t**. In the umbrella example, we can compute the backward message from **Day 2** to **Day 1** by using the `backward` function. The `backward` function has as parameters the object created by the **`HiddenMarkovModel`** class, the evidence in **Day 2** (in our case is **True**), and the initial probabilities of being in state in time t+1. Since no observation is available then it will be [1, 1]. The `backward` function will return a list with the conditional probabilities.

$$P(\mathbf{\mathsf{e}}_{k+1:t} \mid X_{k}) = \sum_{x_{k+1}}P(\mathbf{\mathsf{e}}_{k+1}\mid x_{k+1})P(\mathbf{\mathsf{e}}_{k+2:t} \mid x_{k+1})P(x_{k+1} \mid X_{k})$$

Which we implement in Python below

In [ ]:
def backward(HMM, b, ev):
    sensor_dist = HMM.sensor_dist(ev)
    prediction = element_wise_product(sensor_dist, b)

    return normalize(vector_add(scalar_vector_product(prediction[0], HMM.transition_model[0]),
                                scalar_vector_product(prediction[1], HMM.transition_model[1])))

In [ ]:
b = [1, 1]
backward(hmm, b, ev=True)

Some may notice that the result is not the same as in the book. The main reason is that in the book example for backward alone does not use the normalization step. If we want to normalize the result, one can use the **`normalize()`** helper function.

In order to find the smoothed estimate for raining in **Day k**, we will implement the **`forward_backward()`** function. As in the example in the book, the umbrella is observed in both days and the prior distribution is ```[0.5, 0.5]```,

In [ ]:
pseudocode('Forward-Backward')

You can review the pseudocode to develop your implementation of the **`forward_backward()`** function.

In [ ]:
def forward_backward(HMM, ev):
    """
    [Figure 15.4]
    Forward-Backward algorithm for smoothing. Computes posterior probabilities
    of a sequence of states given a sequence of observations.
    """
    t = len(ev)
    ev.insert(0, None)  # to make the code look similar to pseudo code

    fv = [[0.0, 0.0] for _ in range(len(ev))]
    b = [1.0, 1.0]
    sv = [[0, 0] for _ in range(len(ev))]

    fv[0] = HMM.prior

    ### YOUR CODE HERE

    



    

    ### END CODE
    
    return sv

And we can test the algorithm with the example in the book.

In [ ]:
umbrella_prior = [0.5, 0.5]
umbrella_transition_model = [[0.7, 0.3], [0.3, 0.7]]
umbrella_sensor_model = [[0.9, 0.2], [0.1, 0.8]]
hmm = HiddenMarkovModel(umbrella_transition_model, umbrella_sensor_model, umbrella_prior)
prob = forward_backward(hmm, ev=[T, T])
print ('The probability of raining in Day 0 is {:.2f} and in Day 1 is {:.2f}'.format(prob[0][1], prob[1][1]))


Since HMMs are represented as single variable systems, we can represent the transition model and sensor model as matrices.
The `forward_backward` algorithm can be easily carried out on this representation (as we have done here) with a time complexity of $O({S}^{2} t)$ where $t$ is the length of the sequence and each step multiplies a vector of size $S$ with a matrix of dimensions $S \times S$.

Since the forward pass stores $t$ vectors of size $S$, this makes the auxiliary space requirement equivalent to $O(St)$.



Is there any way we can improve the time or space complexity?



Fortunately, the matrix representation of HMM properties allows us to do so.


If $f$ and $b$ represent the forward and backward messages respectively, we can modify the smoothing algorithm by first
running the standard forward pass to compute $f_{t:t}$ (forgetting all the intermediate results) and then running
backward pass for both $b$ and $f$ together, using them to compute the smoothed estimate at each step.
This optimization reduces auxlilary space requirement to constant (irrespective of the length of the sequence) provided
the transition matrix is invertible and the sensor model has no zeros (which is sometimes hard to accomplish)


Let's look at another algorithm, that carries out smoothing in a more optimized way.

### Fixed-lag Smoothing

The matrix formulation allows us to optimize online smoothing with a fixed lag.

Since smoothing can be done in constant time, there should exist an algorithm whose time complexity is independent of the length of the lag.
For smoothing a time slice $t - d$ where $d$ is the lag, we need to compute $\alpha f_{1:t-d}$ x $b_{t-d+1:t}$ incrementally.


As we already know, the forward equation is


$$f_{1:t+1} = \alpha O_{t+1}{T}^{T}f_{1:t}$$


and the backward equation is


$$b_{k+1:t} = TO_{k+1}b_{k+2:t}$$


where $T$ and $O$ are the transition and sensor models respectively.


For smoothing, the forward message is easy to compute, but there is no simple relation between the backward message of this time step and the one at the previous time step, hence we apply the backward equation $d$ times to get


$$b_{t-d+1:t} = \left ( \prod_{i=t-d+1}^{t}{TO_i} \right )b_{t+1:t} = B_{t-d+1:t}1$$


where $B_{t-d+1:t}$ is the product of the sequence of $T$ and $O$ matrices.


Below we show the pseudocode for `fixed_lag_smoothing`.


In [ ]:
pseudocode('Fixed-Lag-Smoothing')

You will implement fixed-lag smoothing below. 
This algorithm applies `forward` as usual and optimizes the smoothing step by using the equations above.
This optimization could be achieved only because HMM properties can be represented as matrices.


You can use the `vector_to_diagonal`, `matrix_multiplication` and `inverse_matrix` matrix manipulation functions to simplify the implementation.

You should use `normalize` to normalize the output before returning the smoothed probabilities.

In [ ]:
def fixed_lag_smoothing(e_t, HMM, d, ev, t):
    """
    [Figure 15.6]
    Smoothing algorithm with a fixed time lag of 'd' steps.
    Online algorithm that outputs the new smoothed estimate if observation
    for new time step is given.
    """
    ev.insert(0, None)

    T_model = HMM.transition_model
    f = HMM.prior
    B = [[1, 0], [0, 1]]

    ### YOUR CODE HERE








    
    ### END CODE

Here's how we can use `fixed_lag_smoothing` for inference on our umbrella HMM.

In [ ]:
umbrella_transition_model = [[0.7, 0.3], [0.3, 0.7]]
umbrella_sensor_model = [[0.9, 0.2], [0.1, 0.8]]
hmm = HiddenMarkovModel(umbrella_transition_model, umbrella_sensor_model)

Given evidence ```T```, ```F```, ```T```, ```F``` and ```T```, we want to calculate the probability distribution for the fourth day with a fixed lag of 2 days.


Let `e_t = False`

In [ ]:
e_t = F
evidence = [T, F, T, F, T]
fixed_lag_smoothing(e_t, hmm, d=2, ev=evidence, t=4)

In [ ]:
e_t = T
evidence = [T, T, F, T, T]
fixed_lag_smoothing(e_t, hmm, d=1, ev=evidence, t=4)

We cannot calculate probability distributions when $t$ is less than $d$

In [ ]:
fixed_lag_smoothing(e_t, hmm, d=5, ev=evidence, t=4)

As expected, the output is `None`